In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# connect to website
URL = 'https://www.amazon.in/s?k=best+seller+books&i=stripbooks&rh=n%3A1318160031%2Cp_n_feature_three_browse-bin%3A9141482031%2Cp_n_condition-type%3A8609960031&dc&qid=1653302841&rnid=8609959031&sprefix=best+se%2Cstripbooks%2C387&ref=sr_nr_p_n_condition-type_1'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"}

page = requests.get(URL, headers=headers)
soup1 = BeautifulSoup(page.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [3]:
results = soup2.find_all('div', attrs={'data-component-type':'s-search-result'})

In [4]:
len(results)

16

In [5]:
item = results[1]

In [6]:
details_col = item.find('div', attrs={'class':'sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right'})
title = details_col.h2.span.get_text()
title = title.strip()
print(title)

The Alchemist


In [7]:
url = 'https://www.amazon.in' + details_col.h2.a.get('href')
print(url)

https://www.amazon.in/Alchemist-Paulo-Coelho/dp/8172234988/ref=sr_1_2?keywords=best+seller+books&qid=1653311025&refinements=p_n_feature_three_browse-bin%3A9141482031%2Cp_n_condition-type%3A8609960031&rnid=8609959031&s=books&sprefix=best+se%2Cstripbooks%2C387&sr=1-2


In [8]:
price = details_col.find('span', attrs={'class': 'a-price-whole'}).get_text()
price = price.strip()
print(price)

227


In [9]:
author = details_col.find('a', attrs={'class': 'a-size-base a-link-normal s-light-weight-text s-underline-text s-underline-link-text s-link-style s-link-centralized-style'}).get_text()
author = author.strip()
print(author)

Paulo Coelho


In [10]:
book_format = details_col.find('div', attrs={'class': 'a-row a-spacing-mini a-size-base a-color-base'}).a.get_text()
book_format = book_format.strip()
print(book_format)

Paperback


In [11]:
rating = details_col.i.span.get_text()
rating = rating.strip()
print(rating)

4.6 out of 5 stars


In [12]:
ratings_count = (details_col.find('div', attrs={'class':'a-section a-spacing-none a-spacing-top-micro'})).find('span', attrs={'class':'a-size-base s-light-weight-text'}).get_text()
ratings_count = ratings_count.strip()
print(ratings_count)

78,717


In [13]:
def get_record(item):
    
    details_col = item.find('div', attrs={'class':'sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 s-list-col-right'})
    
    title = details_col.h2.span.get_text()
    title = title.strip()
    
    try:
        author = details_col.find('a', attrs={'class': 'a-size-base a-link-normal s-light-weight-text s-underline-text s-underline-link-text s-link-style s-link-centralized-style'}).get_text()
        author = author.strip()
    except AttributeError:
        author = ''
    
    try:
        price = details_col.find('span', attrs={'class': 'a-price-whole'})
        price = price.get_text()
        price = price.strip()
    except AttributeError:
        price = ''
    
    try:
        rating = details_col.i.span.get_text()
        rating = rating.strip()
        rating = rating[0:3]
    except AttributeError:
        rating = ''
    
    try:
        ratings_count = (details_col.find('div', attrs={'class':'a-section a-spacing-none a-spacing-top-micro'})).find('span', attrs={'class':'a-size-base s-light-weight-text'}).get_text()
        ratings_count = ratings_count.strip()
    except AttributeError:
        ratings_count = ''
    
    book_format = details_col.find('div', attrs={'class': 'a-row a-spacing-mini a-size-base a-color-base'}).a.get_text()
    book_format = book_format.strip()

    url = 'https://www.amazon.in' + details_col.h2.a.get('href')
        
    return (title, author, price, book_format, rating, ratings_count, url)

In [14]:
get_record(results[1])

('The Alchemist',
 'Paulo Coelho',
 '227',
 'Paperback',
 '4.6',
 '78,717',
 'https://www.amazon.in/Alchemist-Paulo-Coelho/dp/8172234988/ref=sr_1_2?keywords=best+seller+books&qid=1653311025&refinements=p_n_feature_three_browse-bin%3A9141482031%2Cp_n_condition-type%3A8609960031&rnid=8609959031&s=books&sprefix=best+se%2Cstripbooks%2C387&sr=1-2')

In [15]:
records = []
results = soup2.find_all('div', attrs={'data-component-type':'s-search-result'})
for item in results:
    records.append(get_record(item))

In [16]:
pd.DataFrame(records)

,0,1,2,3,4,5,6
0,As a Man Thinketh,James Allen,69,Paperback,4.4,"8,074",https://www.amazon.in/As-Man-Thinketh-James-Al...
1,The Alchemist,Paulo Coelho,227,Paperback,4.6,"78,717",https://www.amazon.in/Alchemist-Paulo-Coelho/d...
2,That Night: Four Friends. Twenty Years. One Ha...,Nidhi Upadhyay,206,Paperback,4.3,910,https://www.amazon.in/That-Night-Friends-Haunt...
3,You Only Live Once: One for Passion Two for Lo...,Stuti Changle,247,Paperback,4.4,"2,436",https://www.amazon.in/You-Only-Live-Once-Frien...
4,All the Light We Cannot See: The Breathtaking ...,Anthony Doerr,299\n \n ...,Kindle Edition,4.5,"55,520",https://www.amazon.in/All-Light-We-Cannot-See-...
5,About a Girl: Book 1 (Tess Brookes Series),Lindsey Kelk,399,Mass Market Paperback,4.6,602,https://www.amazon.in/About-Girl-Tess-Brookes-...
6,Journey to the Centre of the Earth,Jules Verne,109,Paperback,4.4,"4,435",https://www.amazon.in/Journey-Centre-Earth-Jul...
7,A Gentleman in Moscow: The worldwide bestseller,Amor Towles,240\n \n ...,Kindle Edition,4.5,"38,723",https://www.amazon.in/Gentleman-Moscow-Amor-To...
8,The Best of Paulo Coelho - (5 Book Slip Case),Paulo Coelho,940,Paperback,4.7,177,https://www.amazon.in/Best-Paulo-Coelho-Book-S...
9,The Secret Messenger: The gripping historical ...,Mandy Robotham,312,Paperback,4.5,"3,660",https://www.amazon.in/Secret-Messenger-Mandy-R...


In [17]:
df = pd.DataFrame(columns=['Title', 'Author', 'Price(in ₹)', 'Book Format', 'Rating', 'Ratings Count', 'URL'])

In [18]:
template = 'https://www.amazon.in/s?k=best+seller+books&i=stripbooks&rh=n%3A1318160031%2Cp_n_feature_three_browse-bin%3A9141482031%2Cp_n_condition-type%3A8609960031&dc&qid=1653302841&rnid=8609959031&sprefix=best+se%2Cstripbooks%2C387&ref=sr_nr_p_n_condition-type_1'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"}

In [19]:
for i in range(1, 21):

    URL = template + '&page={}'.format(i)

    page = requests.get(URL, headers=headers)
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    records = []
    results = soup2.find_all('div', attrs={'data-component-type':'s-search-result'})
    
    for item in results:
        records.append(get_record(item))
        
    df_temp = pd.DataFrame(records, columns=['Title', 'Author', 'Price(in ₹)', 'Book Format', 'Rating', 'Ratings Count', 'URL'])
    df = pd.concat([df, df_temp], ignore_index=True)

In [20]:
df

,Title,Author,Price(in ₹),Book Format,Rating,Ratings Count,URL
0,As a Man Thinketh,,69,Paperback,4.4,,https://www.amazon.in/As-Man-Thinketh-James-Al...
1,The Alchemist,,227,Paperback,4.6,,https://www.amazon.in/Alchemist-Paulo-Coelho/d...
2,That Night: Four Friends. Twenty Years. One Ha...,,206,Paperback,4.3,,https://www.amazon.in/That-Night-Friends-Haunt...
3,You Only Live Once: One for Passion Two for Lo...,,247,Paperback,4.4,,https://www.amazon.in/You-Only-Live-Once-Frien...
4,All the Light We Cannot See: The Breathtaking ...,,299\n \n ...,Kindle Edition,4.5,,https://www.amazon.in/All-Light-We-Cannot-See-...
...,...,...,...,...,...,...,...
315,When Love Came Calling,Preeti Shenoy,98,Paperback,4.4,"1,217",https://www.amazon.in/When-Love-Calling-Preeti...
316,Betty: The International Bestseller,Tiffany McDaniel,541,Paperback,4.5,"1,535",https://www.amazon.in/Betty-International-Best...
317,I Have Something to Tell You: The most thought...,Susan Lewis,577\n \n ...,Kindle Edition,4.2,"5,665",https://www.amazon.in/Have-Something-Tell-You-...
318,Eleven Minutes,Paulo Coelho,260,Paperback,4.4,"3,273",https://www.amazon.in/Eleven-Minutes-Paulo-Coe...


In [21]:
df.to_csv('BestSellingContemporaryFiction.csv', index=False)